In [ ]:
{
    "graph": [montgomery],
    "budget": [750],
    "agent": [LambdaParam(segmented_greedy)],
    "agent_params": [{'split_pcts': [round(1 - p, 3), round(p, 3)]} for p in np.arange(0, 1, 0.01)],
    # "budget": [i for i in range(400, 1260, 50)],
    "policy": ["A"],
    "transmission_rate": [0.05],
    "transmission_known": [False],
    "compliance_rate": [-1.0],
    "compliance_known": [False],
    "discovery_rate": [1.0],
    "snitch_rate": [1.0],
    "from_cache": ["c7.json"],
    "trial_id": [i for i in range(5)]
}


graph = GraphParam('montgomery')
budget = 750
agent = LambdaParam(segmented_greedy)
agent_params = 

with open(PROJECT_ROOT / "data" / "SIR_Cache" / from_cache, 'r') as infile:
    j = json.load(infile)

    (S, I1, I2, R) = (j["S"], j["I1"], j["I2"], j["R"])
    # An array of previous infection accounts
    infections = j["infections"]

raw_history = []
state = InfectionState(graph, (S, I1, I2, R), budget, policy, transmission_rate,
                       transmission_known, compliance_rate, compliance_known, discovery_rate, snitch_rate)

while len(state.SIR.I1) + len(state.SIR.I2) != 0:
    to_quarantine = agent(state, **agent_params)
    # Record full history
    if trial_id == 0:
        raw_history.append({
            'state': {
                'S': list(state.SIR.S),
                'I1': list(state.SIR.I1),
                'I2': list(state.SIR.I2),
                'R': list(state.SIR.R),
            },
            'action': list(to_quarantine),
        })
    state.step(to_quarantine)
    infections.append(len(state.SIR.I2))

# Output data to workers and folders

main_out = {
    "id": id,
    "peak": max(infections),
    # Total infections (number of people recovered)
    "total": len(state.SIR.R),
}
aux_out = [id, *infections]

queues["csv_main"].put(main_out)
queues["csv_inf_history"].put(aux_out)

if trial_id == 0:  # Only record 1 entry
    path = path / "data" / str(id)
    path.mkdir(parents=True, exist_ok=True)
    with open(path / "sir_history.json", "w") as f:
        json.dump(raw_history, f)